## Entrenamiento de modelo para letras dinámicas del LSM

En este notebook entrenamos modelos clásicos sobre secuencias dinámicas (videos) correspondientes a letras como J, K, Ñ, Q, X, Z del Lenguaje de Señas Mexicano. Cada secuencia se representa como un vector promedio de sus frames. Se evalúan distintos clasificadores y se guarda el mejor.

## 1. Carga de datos

In [8]:
import pandas as pd
from pyprojroot import here

df = pd.read_csv(here() / "data" / "letters" / "dynamics_sequences.csv")
df.head()

,frames,label
0,['data/letters/dynamics/J/S1-J-perfil-1/frame_...,J
1,['data/letters/dynamics/J/S1-J-perfil-2/frame_...,J
2,['data/letters/dynamics/J/S1-J-perfil-3/frame_...,J
3,['data/letters/dynamics/J/S1-J-perfil-4/frame_...,J
4,['data/letters/dynamics/J/S1-J-perfil-5/frame_...,J


## 2. Vectorización de secuencias (mean pooling)

Cada secuencia (carpeta de frames) es convertida en un solo vector promedio, aplicando redimensionamiento y aplanamiento por frame.

In [9]:
import cv2
import numpy as np
from tqdm import tqdm

def vectorizar_secuencia(frame_paths):
    frames = [cv2.imread(str(here() / f)) for f in frame_paths]
    frames = [f for f in frames if f is not None]
    arr = np.array([cv2.resize(f, (64,64)).flatten() for f in frames])
    return arr.mean(axis=0) if len(arr) > 0 else np.zeros(64*64*3)

X = np.vstack(df['frames'].apply(eval).apply(vectorizar_secuencia).to_list())
y = df['label'].to_numpy()
print(f"Shape de X: {X.shape}")

Shape de X: (620, 12288)


## 3. Reducción de dimensionalidad con PCA

Se aplica PCA después de estandarizar los vectores para reducir la dimensionalidad y mejorar el entrenamiento.

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)
print("Shape original:", X.shape)
print("Shape tras PCA:", X_pca.shape)

Shape original: (620, 12288)
Shape tras PCA: (620, 5)


## 4. División de datos y comparación de clasificadores

In [11]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, stratify=y, random_state=42)

modelos = {
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000)
}

for nombre, modelo in modelos.items():
    scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{nombre}: Accuracy promedio (CV): {scores.mean():.4f}")

SVM: Accuracy promedio (CV): 0.3931
Random Forest: Accuracy promedio (CV): 0.8386
KNN: Accuracy promedio (CV): 0.6734


c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/pre

Logistic Regression: Accuracy promedio (CV): 0.3729


c:\Users\Jamin\miniconda3\envs\signml\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## 5. Evaluación final y guardado del mejor modelo

In [12]:
from sklearn.metrics import classification_report
import joblib

# Entrenar el mejor modelo
mejor_modelo = RandomForestClassifier()
mejor_modelo.fit(X_train, y_train)

# Evaluación
y_pred = mejor_modelo.predict(X_test)
print(classification_report(y_test, y_pred))

# Guardado
joblib.dump(mejor_modelo, here() / "models" / "dynamics_rf.pkl")
joblib.dump(pca, here() / "models" / "pca_gestos.pkl")
joblib.dump(scaler, here() / "models" / "scaler_gestos.pkl")
print("✅ Modelo y transformadores guardados.")

              precision    recall  f1-score   support

           J       0.85      0.85      0.85        20
           K       0.79      0.95      0.86        20
           Q       0.68      0.62      0.65        21
           X       0.80      0.76      0.78        21
           Z       0.95      0.90      0.93        21
           Ñ       1.00      1.00      1.00        21

    accuracy                           0.85       124
   macro avg       0.85      0.85      0.85       124
weighted avg       0.85      0.85      0.84       124

✅ Modelo y transformadores guardados.


## Conclusiones

- Se entrenaron y compararon cuatro clasificadores tradicionales (SVM, Random Forest, KNN y Regresión Logística) usando validación cruzada.
- El modelo con mejor rendimiento fue **Random Forest**, alcanzando una precisión promedio (CV) de **0.8386**.
- La reducción de dimensionalidad mediante PCA fue efectiva, comprimiendo los vectores de 12,288 dimensiones a solo **5 componentes principales**, manteniendo el 95% de la varianza.
- El rendimiento final del modelo Random Forest alcanzó una **accuracy del 85%** en el conjunto de prueba.
- Se observó un buen desempeño general en todas las clases dinámicas, destacando el gesto **Ñ** con una precisión y recall perfectos.
- Se detectaron algunas dificultades en las letras **Q** y **X**, lo cual podría deberse a variabilidad visual o menor diferenciación entre secuencias.
- El modelo final, junto con el PCA y el scaler, fue guardado correctamente para uso posterior.

> Estos resultados muestran que, aunque los clasificadores tradicionales pueden lograr buenos resultados, existe potencial para mejorar el rendimiento con modelos secuenciales como LSTM o CNN 3D que aprovechen mejor la naturaleza temporal de los datos.